<a href="https://colab.research.google.com/github/kindustrii/doiit-text-klassifizierung-rnn/blob/main/doiit_text_klassifizierung_mit_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Unser Neuronales Netz wir Filmbewertungen, die als Text vorliegen, darauf untersuchen ob es sich um eine positive oder um eine negative Bewertung handelt.

Als Trainingsdaten für das Neuronale Netz werden wird den IMDB-Datensatz verwenden. Es gibt mehrere Wege diesen Datensatz zu laden. Wir verwenden eine Variante mittleren Aufwandes und nutzen dafür das Package tensorflow-datasets. Da dieses Package nicht standardmäßig installiert ist, führen wir zunächst die Installation aus und importieren danach die erforderlichen Bibliotheken.

In [ ]:
!pip install -q tensorflow-datasets

In [ ]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

# IMDB-Datensatz laden

Der IMDB-Datensatz besteht aus 50.000 Filmbewertung (in englischer Sprache) mit je 25.000 positiven und negativen. Eine negative Bewertung ist mit einer 0 markiert und eine positive Bewertung ist mit einer 1 markiert.

In [ ]:
# laden des gesamten Datensatzes
dataset, info = tfds.load('imdb_reviews',with_info=True,
                          as_supervised=True)

# Aufteilen des gesamten Datensatzen in Trainings- und Testdaten
# Die Testdaten werden zur Seite gelegt und wir verweden
# diese zum Schluss um das Neuronale Netz zu testen
# Wir teilen in diesem Fall den Datensatz in dem Verhältnis 50/50 auf
train_data, test_data = dataset['train'],dataset['test'] 


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteN1GUDY/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteN1GUDY/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteN1GUDY/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


Sehen wir uns uns mal die ersten 5 Filmbewertung aus den Trainingsdaten an.

In [ ]:
for bewertung,markierung in train_data.take(5):
  print('Bewertung:  {}'.format(bewertung.numpy()))
  print('Markierung: {}\n'.format(markierung.numpy()))

Bewertung:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Markierung: 0

Bewertung:  b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the fil

Das sieht doch ganz gut aus. Für die ersten fünf Filmbewertungen passt die Markierung 0 (negativ) und 1 (positiv) zu dem jeweiligen Text.

Diese Markierungen wurden von Menschen erstellt. D.h. ein Mensch hat sich die Bewertungen durchgelesen und je nachdem ob darin der Film gut oder schlecht bewertet wurde, mit einer 1 oder mit einer 0 markiert. 

Unsere Aufgabe besteht nun darin ein Neuronales Netz zu erstellen, das in der gleichen Weise wie der Mensch eine Filmbewertung bekommt und diese entweder mit einer 0 oder mit einer 1 markiert. Damit es diese Aufgabe erfolgreich durchführen kann, werden wir in das Neuronalen Netz die Trainingsdaten als Pärchen (Bewertung/Markierung) einspeisen. In dem Training wird, vereinfacht gesagt, das NN die Muster von positiven und negativen Bewertungen lernen und später eine unbekannte Filmbewertung auf diese Muster durchsuchen und so die Entscheidung treffen, ob mit einer 1 oder einer 0 Markiert werden soll.

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
# Durchmischen und aufteilen des gesamten Datensatzes mit 50.000 Filmbewertungen
# in Trainings- und Testdaten  
train_data = train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Bei der Aufteilung des Datensatzes haben wir noch zusätzlich sogenannte Batches
# erstellt. Ein Batch ist im Prinzip ein Bündel, in dem wir jeweils 64 einzelne
# Filmbewertungen vereinen
for bewertung, markierung in train_data.take(1):
  print('Bewertung: {}\n'.format(bewertung.numpy()[:3]))
  print('Markierung: {}\n'.format(markierung.numpy()[:3]))

Bewertung: [b"This is one to watch a few times. The excellent writing shows they had to have lived this story or know someone whom did because they nailed it. Freebird made me relive and laugh at my misspent youth. The title was a Great choice. Great film, setting, story, soundtrack and characters. It's a biker flick but would be a shame to pigeon hole it that way. Funny to the bone, kinda like Trailer Park Boys in the U.K. If you've never seen TPB, make a point to if you like this film. You will thank me. I hope to see more of these characters in other films. Sequel? Could be done. There's a whole lot more of the world I would like to see through their eyes."
 b"Having to have someone hold your hand whenever walk up or down stairs? Having others taste your food before you eat it? Facing an over-bearing mother? These are only a few of the obstacles which the young Victoria has to deal with in this film (there's also the various power struggles going on, as well as attempts on her life)

# Den Text codieren

Die Daten (Filmbewertungen), die vom Neuronalen Netz verarbeitet werden müssen stehen uns in Form von Text zur Verfügung. Bevor diese ans NN übergeben werden können, muss der Text vorverarbeitet und in eine bestimmte Form gebracht werden. Wir verwenden hierfür die sogenannte Wort Vektorisierung, die wir als Verarbeitungsschicht vor die Input-Schicht des Neuronalen Netzes setzen.

Mit dieser Technik erstellen wir zunächst einmal eine Vokabelliste. Dafür werden alle 50.000 Filmbewertungen durchlaufen und jedes einzelne Wort das vorkommt, einmalig in die Liste aufgenommen. Jedes Wort, dass nun in der Vokabelliste steht, erhält dann eine eigene Zahl zugewiesen, sodass jeder Satz mithilfe dieser List in eine Zahlenfolge übersetzt werden kann.

Hier ein Beispiel zur Verdeutlichung.

Unser **Satz** lautet: kindustrii ist super, super cool. 

Daraus entsteht die **Vokabelliste**: [kindustrii, ist, super, cool]. 

Jedem Wort in der Vokabelliste wird **eine Zahl zugeordnet**: {'kindustrii': 0, 'ist': 1, 'super': 2, 'cool': 3}

Übersetzer Satz, bzw. **Vektor**: [0, 1, 2, 2, 3]



In [ ]:
VOCAB_SIZE = 1000
# Der Encoder durchläuft alle Filmbewertungen und sammelt alle Wörter ein
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE
)
encoder.adapt(train_data.map(lambda text, label: text))

In [ ]:
# Wir sehen uns 50 Wörter der Vokabelliste an
vocab = np.array(encoder.get_vocabulary())
vocab[500:550]

array(['starts', 'son', 'kill', 'game', 'act', 'sometimes', 'side',
       'viewer', 'town', 'horrible', 'parts', 'car', 'actress', 'soon',
       'child', 'ones', 'eyes', 'expect', 'obviously', 'flick',
       'themselves', 'directed', 'thinking', 'heart', 'art', 'brilliant',
       'stories', 'ill', 'decent', 'highly', 'run', 'feeling', 'myself',
       'genre', 'late', 'blood', 'stuff', 'fight', 'says', 'close',
       'took', 'city', 'except', 'cannot', 'heard', 'hand', 'leave',
       'killed', 'kid', 'matter'], dtype='<U14')

In [ ]:
# Hier sehen wir uns mal drei Filmbewertungen an, die als Zahlen-Vektor dargestellt sind
# Hinweis: Alle Filmbewertungen werden auf dieselbe Länge gebracht, in dem kurze
# Texte hinten mit 0en aufgefüllt werden

encoded_example = encoder(bewertung)[:3].numpy()
encoded_example

array([[ 11,   7,  29, ...,   0,   0,   0],
       [252,   6,  26, ...,   0,   0,   0],
       [210,  36,   2, ...,   0,   0,   0]])

Jede der drei oberen Zeilen repräsentiert eine Filmbewertung. An den hinteren Nullen erkennt man, dass die Bewertungen weniger Wörter enthalten als die längste aller Bewertungen und deshalb aufgefüllt werden. Diese Nullen werden später im Inneren des Neuronalen Netzes herausgefiltert, um das Ergebnis nicht zu beeinflussen.

Vergewissern wir uns noch kurz, dass die drei oberen Vektoren tatsächlich gleich viele Elemente enthalten.

In [ ]:
for i in range(3):
  print('Laenge der Filmbewertung {}: {}'.format(i,len(encoded_example[i])))

Laenge der Filmbewertung 0: 916
Laenge der Filmbewertung 1: 916
Laenge der Filmbewertung 2: 916


# Das Neuronale Netz erstellen

In der Code-Zeile [12] wird das gesamte Neuronale Netz erstellt, dass insgesamt aus 5 Schichten besteht. In der ersten Schicht befindet sich wie schon erwähnt der Encoder, den wir weiter oben erstellt haben, um die Texte in Vektoren zu übersetzen. Die letzte Schicht enthält ein einziges Neuron, das uns das Ergebnis (0 oder 1) liefern wird.

Nachdem der Encoder die Vektoren gebildet hat, werden diese an das Embedding Layer übergeben. In dieser Schicht des Netzes wird gelernt welche Wörter die gleiche oder eine ähnliche Bedeutung haben. Wie zum Beispiel die beiden Wörter good und great. Hierfür wird jedes einzelne Wort als eigener Vektor dargestellt.

Die als LSTM bezeichnete Schicht befindet sich in der Mitte des Netzes und konvertiert die Vektorsequenzen in einen einzelnen Vektor. In der vorletzten Schicht wird dieser Vektor in eine einzelne Zahl (0 oder 1), die dann in der letzten Schicht herausgegeben wird. 

In [ ]:
model = tf.keras.Sequential()
model.add(encoder)
model.add(tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()),
                                    output_dim=64,
                                    mask_zero=True))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(1))

In der nächsten Code-Zeile testen wir das Neuronale Netz bevor wir es trainiert haben. Dafür lassen wir es eine kurze und eindeutig positive Filmbewertung analysieren.

Als Ergebnis liefert uns das Neuronale Netz ein 0. Das ist offensichtlich falsch, denn eine 0 steht für eine negative Filmbewertung. Wir testen das Netz noch einmal, nachdem wir es trainiert haben und hoffentlich erhalten wir dann auch das richtige Ergebnis.

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[-0.0006106]


# Das Neuronale Netz trainieren

In der Code-Zeile [14] legen wir noch einige Parameter für das Training fest und in der Code-Zeile [15] wird es gestartet und durchgeführt. Wir führen das Training für 10 Epochen aus, wobei in jeder Epoche der gesamte Trainingsdatensatz durch das Neuronale Netz geleitet wird. Das Neuronale Netz bekommt 25.000 Filmbewertungen als insgesamt 10 Mal zu sehen. 

Nach jeder Epoche wird zusätzlich geprüft wie gut das Netz bereits trainiert ist. Die Werte accuracy und val_accuracy die nach jeder Epoche während dem Training ausgegeben werden zeigen an wie genau das Netz die Vorhersagen macht. Je näher die Werte an der 1 liegen, desto besser ist es trainiert.

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,epochs=10,
                    validation_data=test_data,
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 33s 85ms/step - loss: 0.6362 - accuracy: 0.5739 - val_loss: 0.4827 - val_accuracy: 0.7818
Epoch 2/10
391/391 [==============================] - 31s 80ms/step - loss: 0.4138 - accuracy: 0.8151 - val_loss: 0.3758 - val_accuracy: 0.8224
Epoch 3/10
391/391 [==============================] - 32s 81ms/step - loss: 0.3392 - accuracy: 0.8539 - val_loss: 0.3470 - val_accuracy: 0.8562
Epoch 4/10
391/391 [==============================] - 32s 81ms/step - loss: 0.3188 - accuracy: 0.8623 - val_loss: 0.3372 - val_accuracy: 0.8604
Epoch 5/10
391/391 [==============================] - 32s 82ms/step - loss: 0.3103 - accuracy: 0.8660 - val_loss: 0.3243 - val_accuracy: 0.8615
Epoch 6/10
391/391 [==============================] - 32s 81ms/step - loss: 0.3043 - accuracy: 0.8697 - val_loss: 0.3243 - val_accuracy: 0.8620
Epoch 7/10
391/391 [==============================] - 32s 81ms/step - loss: 0.2995 - accuracy: 0.8720 - val_loss: 0.3225 - val_accuracy:

Das Neuronale Netz wird zum Schluss evaluiert, indem alle 25.000 Filmbewertungen, die wir anfangs zur Seite gelegt haben bewertet werden. Wir interessieren uns im Moment für den Wert Test Accuracy, der bei der Evaluierung bei ca. 0.86 liegt. Das bedeutet, dass das Neuronale Netz bei 86% der 25.000 Filmbewertungen richtig erkannt hat, ob es sich um eine positive oder negative Bewertung handelt.

Dafür, dass wir ein recht einfaches Netz verwendet haben und das Training gerade mal 7 Minuten gedauert hat, ist es ein sehr gutes Ergebnis. Allerdings lässt es sich noch verbessern, wenn noch Anpassungen am Netz und an den Parametern im Training vorgenommen werden.

In [ ]:
test_loss,test_acc = model.evaluate(test_data)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 16s 41ms/step - loss: 0.3231 - accuracy: 0.8633
Test Loss: 0.3230961859226227
Test Accuracy: 0.863319993019104


Abschließend testen wir nun das trainierte Netz mit derselben Filmbewertung wie vorher, bei der wir ein falsches Ergebnis erhalten haben. Dieses Mal erhalten wir das Ergebnis 0.78. Das liegt deutlich näher an der 1 als an der 0 und wird entsprechend als richtig gewertet. Den Wert 0.78 interpretieren wir so, dass sich das Neuronale Netz zu 78% sicher ist, dass es sich um eine positive Filmbewertung handelt. 

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.7818757]
